In [1]:
%load_ext sql
import pandas as pd
import numpy as np

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

![dbSchema](159739A3Schema.jpg)

### High Level Description of [insert Name] Database

#### Players
Stores information about tennis players, including their unique PlayerID, Name, Ranking, Points, and details about whether they have wealthy parents and if their parents are mentioned on Wikipedia.

#### Matches
This table contains details about individual tennis matches, including MatchID, LocationID, TournamentID, CourtID, SurfaceID, RoundID, BestOf (the number of sets required to win the match), WLRank (the opponent's ranking), WLsets (the number of sets won and lost), Comment, AvgWL (average winning likelihood), and Win (indicating if the player won the match).

#### Locations
Represents the locations where tennis matches take place. It includes LocationID and Name.

#### Tournaments
Stores information about tennis tournaments; TournamentID, Tier information (the level or category of the tournament, such as International, Grand Slam, etc.) and Name.

#### Courts
Describes the types of courts used in tennis matches. It includes CourtID and Type (e.g., Outdoor, Indoor).

#### Surfaces
Contains information about the playing surfaces used in tennis matches, such as SurfaceID and Type (e.g., Hard, Clay, Grass).

#### Rounds
Represents the different rounds or stages within a tennis tournament. It includes RoundID and Name (e.g., 1st Round, Quarterfinals, Finals).

#### PlayerMatches
This table establishes the one-to-many relationships between players and matches. It includes PlayerMatchID (a unique identifier for the relationship), PlayerID (referencing the Players table), and MatchID (referencing the Matches table.

## Connect to DB and Create Tables

In [3]:
%sql sqlite:///./158739Assignment3

In [4]:
%%sql sqlite:///./158739Assignment3

DROP TABLE IF EXISTS Courts;
CREATE TABLE "Courts" (
"CourtID" INTEGER NOT NULL UNIQUE,
"Type" TEXT,
PRIMARY KEY("CourtID" AUTOINCREMENT)
)

Done.
Done.


[]

In [5]:
%%sql sqlite:///./158739Assignment3

DROP TABLE IF EXISTS PlayerDetails;

CREATE TABLE PlayerDetails (
    "PlayerDetailsID"	INTEGER NOT NULL UNIQUE,
    "PlayerID"	INTEGER,
    "WealthyParents"	INTEGER,
    "ParentOnWiki"	INTEGER,
    FOREIGN KEY("PlayerID") REFERENCES "Players"("PlayerID")
);

Done.
Done.


[]

In [6]:
%%sql sqlite:///./158739Assignment3

DROP TABLE IF EXISTS Locations;

CREATE TABLE "Locations" (
"LocationID" INTEGER NOT NULL UNIQUE,
"Name" TEXT,
PRIMARY KEY("LocationID" AUTOINCREMENT)
)

Done.
Done.


[]

In [7]:
%%sql sqlite:///./158739Assignment3

DROP TABLE IF EXISTS Matches;
    CREATE TABLE "Matches" (
"MatchID"	INTEGER NOT NULL UNIQUE,
"LocationID"	INTEGER,
"TournamentID"	INTEGER,
"CourtID"	INTEGER,
"SurfaceID"	INTEGER,
"RoundID"	INTEGER,
"BestOf"	INTEGER,
"WLRank"	REAL,
"WLsets"	INTEGER,
"Comment"	TEXT,
"AvgWL"	REAL,
"Win"	BOOLEAN,
"PlayerID"	INTEGER,
        
PRIMARY KEY("MatchID" AUTOINCREMENT)
)

Done.
Done.


[]

In [8]:
%%sql sqlite:///./158739Assignment3

DROP TABLE IF EXISTS PlayerMatches;
CREATE TABLE "PlayerMatches" (
"PlayerMatchID"	INTEGER NOT NULL UNIQUE,
"PlayerID"	INTEGER,
"MatchID"	INTEGER,
FOREIGN KEY("PlayerID") REFERENCES "Players"("PlayerID"),
FOREIGN KEY("MatchID") REFERENCES "Matches"("MatchID"),
PRIMARY KEY("PlayerMatchID" AUTOINCREMENT)
)

Done.
Done.


[]

In [9]:
%%sql sqlite:///./158739Assignment3

DROP TABLE IF EXISTS Players;
CREATE TABLE "Players" (
"PlayerID"	INTEGER NOT NULL UNIQUE,
"Name"	TEXT,
"Ranking"	INTEGER,
"WealthyParents"	BOOLEAN,
PRIMARY KEY("PlayerID" AUTOINCREMENT)
)

Done.
Done.


[]

In [10]:
%%sql sqlite:///./158739Assignment3

DROP TABLE IF EXISTS Rounds;
CREATE TABLE "Rounds" (
"RoundID"	INTEGER NOT NULL UNIQUE,
"Name"	TEXT,
PRIMARY KEY("RoundID" AUTOINCREMENT)
)

Done.
Done.


[]

In [11]:
%%sql sqlite:///./158739Assignment3

DROP TABLE IF EXISTS Surfaces;
CREATE TABLE "Surfaces" (
"SurfaceID"	INTEGER NOT NULL UNIQUE,
"Type"	TEXT,
PRIMARY KEY("SurfaceID" AUTOINCREMENT)
)

Done.
Done.


[]

In [12]:
%%sql sqlite:///./158739Assignment3

DROP TABLE IF EXISTS Tournaments;

CREATE TABLE "Tournaments" (
"TournamentID"	INTEGER NOT NULL UNIQUE,
"Name"	TEXT,
"Tier"	TEXT,
PRIMARY KEY("TournamentID" AUTOINCREMENT)
)

Done.
Done.


[]

## Import data into tables

In [13]:
# import CSV into a dataframe
# or get top_players from csv, to reduce API calls
player_odds_df = pd.read_csv("all_matches_player_odds.csv", index_col=0)
pd.set_option('display.max_columns', None)
player_odds_df.head()


,WTA,Location,Tournament,Date,Tier,Court,Surface,Round,Best of,WinLoseName,WLRank,WPts,WLsets,Comment,AvgWL,ATP,Series,win,LPts,Name,Rank,wealthy_parents,parent_on_wiki
0,1.0,Auckland,ASB Classic,31/12/2018,International,Outdoor,Hard,1st Round,3.0,Puig M.,53.0,1010.0,2.0,Completed,1.37,0.0,0,True,0.0,NaN,0.0,Flase,False
1,1.0,Auckland,ASB Classic,31/12/2018,International,Outdoor,Hard,1st Round,3.0,Davis L.,173.0,340.0,2.0,Completed,1.50,0.0,0,True,0.0,Lauren Davis,58.0,False,True
2,1.0,Auckland,ASB Classic,31/12/2018,International,Outdoor,Hard,1st Round,3.0,Kuzmova V.,50.0,1052.0,2.0,Completed,1.62,0.0,0,True,0.0,NaN,0.0,Flase,False
3,1.0,Auckland,ASB Classic,31/12/2018,International,Outdoor,Hard,1st Round,3.0,Bouchard E.,87.0,692.0,2.0,Completed,1.53,0.0,0,True,0.0,NaN,0.0,Flase,False
4,1.0,Auckland,ASB Classic,31/12/2018,International,Outdoor,Hard,1st Round,3.0,Sorribes Tormo S.,86.0,700.0,2.0,Completed,2.35,0.0,0,True,0.0,NaN,0.0,Flase,False


In [14]:
# populate players table

unique_Name = pd.DataFrame(player_odds_df['WinLoseName'].drop_duplicates())
names = unique_Name['WinLoseName'].tolist()

%sql DELETE FROM Players

for name in names:
    rank = player_odds_df.loc[player_odds_df['WinLoseName'] == name, 'Rank'].values[0]
    wealthyparents = player_odds_df.loc[player_odds_df['WinLoseName'] == name, 'wealthy_parents'].values[0]
    %sql INSERT INTO Players (Name, Ranking, WealthyParents) VALUES (:name, :rank, :wealthyparents)


 * sqlite:///./158739Assignment3
0 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.


 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.


1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3


 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.


 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.


1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3


In [16]:
# populate matches table

unique_avgwl = pd.DataFrame(player_odds_df['AvgWL'].drop_duplicates())
                                
avgwl = unique_avgwl['AvgWL'].tolist()

%sql DELETE FROM Matches
    
for avg in avgwl:
    sets = player_odds_df.loc[player_odds_df['AvgWL'] == avg, 'WLsets'].values[0]
    bests = player_odds_df.loc[player_odds_df['AvgWL'] == avg, 'Best of'].values[0]
    comments = player_odds_df.loc[player_odds_df['AvgWL'] == avg, 'Comment'].values[0]
    wins = player_odds_df.loc[player_odds_df['AvgWL'] == avg, 'win'].values[0]
    %sql INSERT INTO Matches (AvgWL, WLsets, BestOf, Comment, Win) VALUES (:avg, :sets, :bests, :comments, :wins)

 * sqlite:///./158739Assignment3
0 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.


1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3


 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.


 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.


1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3


1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3


 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.


In [17]:
# populate tournaments table

unique_tournaments = pd.DataFrame(player_odds_df['Tournament'].drop_duplicates())
tournaments = unique_tournaments['Tournament'].tolist()

%sql DELETE FROM Tournaments

for tournament in tournaments:
    tier = player_odds_df.loc[player_odds_df['Tournament'] == tournament, 'Tier'].values[0]
    %sql INSERT INTO Tournaments (Name, Tier) VALUES (:tournament, :tier)

 * sqlite:///./158739Assignment3
0 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.


 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.


In [18]:
# populate courts table
1
unique_values_df = pd.DataFrame(player_odds_df['Court'].drop_duplicates())
unique_ids = pd.RangeIndex(start=1, stop=len(unique_values_df) + 1)
unique_values_df.insert(0, 'ID', unique_ids)

data = unique_values_df.values.tolist()

%sql DELETE FROM Courts

for row in data:
    trow = tuple(row)
    sql = "INSERT INTO Courts VALUES {}".format(trow)  
    %sql $sql

 * sqlite:///./158739Assignment3
0 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.


In [19]:
# populate Locations table
    
unique_location = pd.DataFrame(player_odds_df['Location'].drop_duplicates())

locations = unique_location['Location'].tolist()

%sql DELETE FROM Locations

for location in locations:
    %sql INSERT INTO Locations (Name) VALUES (:location)

 * sqlite:///./158739Assignment3
0 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.


In [21]:
# populate Surfaces table
unique_values_df = pd.DataFrame(player_odds_df['Surface'].drop_duplicates())
unique_ids = pd.RangeIndex(start=1, stop=len(unique_values_df) + 1)
unique_values_df.insert(0, 'ID', unique_ids)

data = unique_values_df.values.tolist()

%sql DELETE FROM Locations

for row in data:
    trow = tuple(row)
    sql = "INSERT INTO Surfaces VALUES {}".format(trow)  
    %sql $sql

 * sqlite:///./158739Assignment3
135 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.


In [22]:
# populate Rounds table
unique_values_df = pd.DataFrame(player_odds_df['Round'].drop_duplicates())
unique_ids = pd.RangeIndex(start=1, stop=len(unique_values_df) + 1)
unique_values_df.insert(0, 'ID', unique_ids)

data = unique_values_df.values.tolist()

%sql DELETE FROM Locations

for row in data:
    trow = tuple(row)
    sql = "INSERT INTO Rounds VALUES {}".format(trow)  
    %sql $sql

 * sqlite:///./158739Assignment3
0 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.


### analysis
1. Query to return win ratio of a player
2. Query to give each players average odds for winning a match
3. Query to return how many matches each player played
4. Query to list all grand-slams played by a specific player
5. Query to return the player with the most wins per surface
6. Query to return the best player (highest win percentage).

In [ ]:
# Win ratio. 

def win_ratio(tennis_player_name):
    
    tennis_player_ID = %sql SELECT PlayerID FROM Players WHERE name = :tennis_player_name
    
    return %sql SELECT COUNT(*) AS total_rows,
           SUM(CASE WHEN wins = true THEN 1 ELSE 0 END) AS true_rows,
           SUM(CASE WHEN wins = true THEN 1 ELSE 0 END) / COUNT(*) AS proportion_true
    FROM PlayerMatches
    WHERE PlayerID = :tennis_player_ID
        

In [ ]:
# return average odds

%sql SELECT Name, AVG(avgWL) AS average_betting_odds
FROM (
  SELECT *
  FROM PlayerMatches
  LEFT JOIN Players ON PlayerMatches.playerID = Players.PlayerId;
)
GROUP BY Name;
